In [1]:
import tensorflow as tf

/root/miniconda3/envs/tensorflow/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/root/miniconda3/envs/tensorflow/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/root/miniconda3/envs/tensorflow/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
import numpy as np
import os
import time
import matplotlib.pyplot as plt
from random import randint
from PIL import Image

In [37]:
STARTER_LR = 1e-4
BATCH_SIZE = 8
NUM_CLASSES = 10
MAX_STEPS = 1000
IMAGE_SIZE = 28
OUTPUT_NAMES = ["fc2/Relu"]

In [4]:
def WeightsVariable(shape, name = 'weights'):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.1, name=name))

def BiasVariable(shape, name = 'biases'):
    return tf.Variable(tf.constant(0.1, shape=[shape], name='biases'))

def Conv2d(x, W, B, stride = 1, padding = 'VALID', activation_fun = True):
    filter_size = W.get_shape().as_list()
    pad_size = filter_size[0] // 2
    pad_mat = np.array([[0, 0], [pad_size, pad_size], [pad_size, pad_size], [0, 0]])
    
    x = tf.pad(x, pad_mat)
    x = tf.nn.conv2d(x, W, strides=[1, stride, stride, 1], padding = padding)
    x = tf.nn.bias_add(x, B)
    
    if(activation_fun == True):
        return tf.nn.relu(x)
    else:
        return x

def MaxPool2d(x, k = 2):
    return tf.nn.max_pool(x, ksize = [1, k, k, 1], strides = [1, k, k, 1], padding = 'VALID')

In [35]:
def network(images, channels = 1):
    #COONVOLUTION 1
    with tf.name_scope('Conv1'):
        weights = WeightsVariable([3, 3, channels, 32])
        bias = BiasVariable(32)
        conv1 = Conv2d(images, weights, bias)
        
    with tf.name_scope('Conv2_m'):
        weights = WeightsVariable([3, 3, 32, 64])
        bias = BiasVariable(64)
        conv2 = Conv2d(conv1, weights, bias)
        pool1 = MaxPool2d(conv2)
        
    with tf.name_scope('Conv3'):
        weights = WeightsVariable([3, 3, 64, 64])
        bias = BiasVariable(64)
        conv3 = Conv2d(pool1, weights, bias)
    
    with tf.name_scope('Conv4_m'):
        weights = WeightsVariable([3, 3, 64, 128])
        bias = BiasVariable(128)
        conv4 = Conv2d(conv3, weights, bias)
        pool2 = MaxPool2d(conv4)
    
    with tf.name_scope('flatten'):
        flat = tf.layers.flatten(pool2)
    
    input_flat_shape = pool2
    
    with tf.name_scope('fc1'):
        weights = WeightsVariable([6272, 1024])
        biases = BiasVariable(1024)
        fc1 = tf.nn.relu(tf.matmul(flat, weights) + biases)

    with tf.name_scope('fc2'):
        weights = WeightsVariable([1024, 10])
        biases = BiasVariable(10)
        fc2 = tf.nn.relu(tf.matmul(fc1, weights) + biases)
    print(fc2)    
    return fc2

In [6]:
def loss_metrics(logits, labels):
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = labels, 
                                                                   logits = logits, 
                                                                   name = 'softmax')
    return tf.reduce_mean(cross_entropy, name = 'softmax_mean')

In [7]:
#returns the optimizer by taking the loss
def training(loss):
    tf.summary.scalar('loss', loss)
    global_step = tf.Variable(0, name = 'global_step', trainable = False)
    
    learning_rate = tf.train.exponential_decay(STARTER_LR, 
                                               global_step = global_step, 
                                               decay_steps = 1000, 
                                               decay_rate = 0.7, 
                                               staircase = True)

    tf.summary.scalar('learning_rate', learning_rate)
    optimizer = tf.train.MomentumOptimizer(learning_rate = learning_rate, momentum = 0.9)
    train_op = optimizer.minimize(loss, global_step = global_step)
    return train_op
    

In [8]:
def evaluation(logits, labels):
    correct = tf.nn.in_top_k(logits, labels, k = 1)
    return tf.reduce_sum(tf.cast(correct, tf.int32))

In [22]:
def do_eval(sess, eval_correct, image_placeholder, labels_placeholder, data_set, summary):
    true_count = 0
    steps_per_epoch = data_set.num_examples // BATCH_SIZE
    num_examples = steps_per_epoch * BATCH_SIZE
    
    for steps in range(steps_per_epoch):
        feed_dict = fill_feed_dict(data_set, image_placeholder, labels_placeholder)
        log, correctness = sess.run([summary, eval_correct], feed_dict = feed_dict)
        true_count += correctness
    
    precision = float(true_count) / num_examples
    tf.summary.scalar('Precision', tf.constant(precision))
    print('Num examples %d, Num Correct: %d Precisiokn @ 1: %0.04f' %
          (num_examples, true_count, precision))
    
    return log    

In [10]:
def placeholder_inputs(batch_size, shape = [28, 28, 1]):
    image_placeholder = tf.placeholder(tf.float32, shape = (None, shape[0], shape[1], shape[2]))
    label_placeholder = tf.placeholder(tf.int32, shape = (None))
    return image_placeholder, label_placeholder

In [11]:
def fill_feed_dict(data_set, image_placeholder, label_placeholder):
    images_feed, labels_feed = data_set.next_batch(BATCH_SIZE)
    feed_dict = {
        image_placeholder: np.reshape(images_feed, (-1, 28, 28, 1)),
        label_placeholder: labels_feed
                }
    return feed_dict

In [50]:
def run_training(Dataset):
    with tf.Graph().as_default():
        images_pl, labels_pl = placeholder_inputs(BATCH_SIZE)
        logits = network(images_pl)
        loss = loss_metrics(logits = logits, labels = labels_pl)
        train_op = training(loss)
        eval_correct = evaluation(logits, labels_pl)
        
        summary = tf.summary.merge_all()
        
        init = tf.global_variables_initializer()
        saver = tf.train.Saver()
        gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.5) #OPTIONAL
        sess = tf.Session(config=tf.ConfigProto(gpu_options = gpu_options))
        
        summary_writer = tf.summary.FileWriter("/tmp/tf/eg/log", 
                                               graph = tf.get_default_graph())
        test_writer = tf.summary.FileWriter("tmp/tf/eg/validation/log", 
                                            graph = tf.get_default_graph())
        
        sess.run(init)
        for steps in range(MAX_STEPS):
            start_time = time.time()
            feed_dict = fill_feed_dict(Dataset.train, images_pl, labels_pl)
            
            _, loss_value = sess.run([train_op, loss], feed_dict = feed_dict)
            duration = time.time() - start_time
            
            if (steps%100 == 0):
                print('Step %d: loss = %.2f (%.3f sec)' % (steps, loss_value, duration))
                summary_str = sess.run(summary, feed_dict = feed_dict)
                summary_writer.add_summary(summary_str, steps)
                summary_writer.flush()
                
            if (steps + 1) % 1000 == 0 or (steps + 1) == MAX_STEPS:
                checkpoint_file = os.path.join("model", "model.ckpt")
                saver.save(sess, checkpoint_file, global_step=steps)
                print('Validation Data Eval:')
                log = do_eval(sess,
                              eval_correct,
                              images_pl,
                              labels_pl,
                              Dataset.validation,
                              summary)
                test_writer.add_summary(log, steps)
                
        graphdef = tf.get_default_graph().as_graph_def()
        frozen_graph = tf.graph_util.convert_variables_to_constants(sess,
                                                                    graphdef,
                                                                    OUTPUT_NAMES)
        return tf.graph_util.remove_training_nodes(frozen_graph)

In [41]:
MNIST_DATASETS = tf.contrib.learn.datasets.load_dataset("mnist")

Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz


In [44]:
tf_model = run_training(MNIST_DATASETS)

Tensor("fc2/Relu:0", shape=(?, 10), dtype=float32)
Step 0: loss = 9.01 (0.015 sec)
Step 100: loss = 2.30 (0.002 sec)
Step 200: loss = 2.30 (0.003 sec)
Step 300: loss = 2.30 (0.002 sec)
Step 400: loss = 2.30 (0.002 sec)
Step 500: loss = 2.30 (0.002 sec)
Step 600: loss = 2.30 (0.003 sec)
Step 700: loss = 2.30 (0.002 sec)
Step 800: loss = 2.30 (0.002 sec)
Step 900: loss = 2.30 (0.002 sec)
Validation Data Eval:
Num examples 5000, Num Correct: 4998 Precisiokn @ 1: 0.9996
INFO:tensorflow:Froze 12 variables.
Converted 12 variables to const ops.


In [61]:
graph = tf.GraphDef()
graph.ParseFromString(tf_model.SerializeToString())

with tf.Graph().as_default() as graph:
        # The name var will prefix every op/nodes in your graph
        # Since we load everything in a new graph, this is not needed
    tf.import_graph_def(graph, name="prefix")
    
    

TypeError: graph_def must be a GraphDef proto.